In [1]:
import pandas as pd
import numpy as np
import re
from function_file import clean_names


C:\Users\K\AppData\Local\Temp\ipykernel_5612\1102577416.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
raw_bird = pd.read_excel(r"data_in\seabirds.xls", sheet_name= "Bird data by record ID")
raw_ship = pd.read_excel(r"data_in\seabirds.xls", sheet_name= "Ship data by record ID")
# Bird data by record ID

In [3]:
clean_ship = raw_ship.rename(columns = clean_names).copy()
clean_ship.columns

Index(['record', 'record_id', 'date', 'time', 'lat', 'long', 'ew', 'sact',
       'speed', 'sdir', 'cld', 'prec', 'wspeed', 'wdir', 'atmp', 'aprs',
       'sste', 'stmp', 'sal', 'depth', 'obs', 'csmeth', 'month', 'seasn',
       'long360', 'latcell', 'longecell'],
      dtype='object')

In [4]:

col_to_rename = {'species_common_name_taxon_age__sex__plumage_phase': 'common_name',
                'species__scientific_name_taxon_age_sex___plumage_phase': 'sci_name',
                'species_abbreviation': 'abbreviation'}

# Clean Column Names
clean_bird = raw_bird.rename(columns = clean_names).copy()
# Replace a couple of key columns with assigned names
clean_bird.rename(columns = col_to_rename, inplace = True)
clean_bird.columns

Index(['record', 'record_id', 'common_name', 'sci_name', 'abbreviation', 'age',
       'wanplum', 'plphase', 'sex', 'count', 'nfeed', 'ocfeed', 'nsow',
       'ocsow', 'nsoice', 'ocsoice', 'ocsoshp', 'ocinhd', 'nflyp', 'ocflyp',
       'nacc', 'ocacc', 'nfoll', 'ocfol', 'ocmoult', 'ocnatfed'],
      dtype='object')

In [5]:

df_bird = clean_bird[["record_id", "common_name", "sci_name", "abbreviation", "count"]].copy()
           

In [6]:
# Merge df_ship and df_bird by record_ID
df = pd.merge(clean_ship, df_bird,how = "left", on = "record_id")

In [7]:

# Which bird had the highest total count?
df_bird_count = df.groupby(["abbreviation"])["count"].sum()
df_bird_count.sort_values(ascending= False)

abbreviation
PUFTEN        982553.0
PACSP         326499.0
PUFGRI        174329.0
PUFGAV        113333.0
PUFBUL         64750.0
                ...   
THABUL IMM         1.0
MACGIG DRK         1.0
LARNOV IMM         0.0
ANOMIN             0.0
STEHIR IMM         0.0
Name: count, Length: 320, dtype: float64

In [8]:
# Which bird had the highest total count above a latitude of -30?

df_above_lat = df.query('lat >= -30').groupby(["abbreviation"])["count"].sum()

df_above_lat.sort_values(ascending = False).idxmax()

'PUFPAC'

In [9]:
# How many different types of birds were only ever seen in groups of 1?
df_group_1 = df.loc[df["count"]== 1]
df_group_1_count = df_group_1["abbreviation"].nunique()
df_group_1_count


297

In [16]:
# How many penguins were seen? (Hint: there are many types of penguin)

df_penguin = df.dropna(subset = ["common_name"]).copy()
# String to lower case
df_penguin["common_name"] = df_penguin["common_name"] .str.upper()

df_penguin = df_penguin[df_penguin["common_name"].str.contains("PENGUIN") ]

df_penguin_count = df_penguin["count"].sum()
df_penguin_count

158.0